In [4]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json

In [5]:
# Hyperparameters
player_names = ["Thomas", "Emily", "Benjamin", "Sophia", "Victoria"]
roles = [0,0,0,1,1]

model = "gpt-3.5-turbo-16k" # "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf"
}

In [3]:
n_games = 15

for game in range(n_games):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Init all the players
    players : Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=["The werewolfs (including you) are [" + ",".join(werewolf_names) + "]."] if name in werewolf_names else [])
        players[name.lower()].init_player(players=player_names)

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []

    # Debate stage
    while rounds < 20 and not stop:
        rounds += 1
        next_to_speak = get_next_speaker(conversation_history, player_names).lower()

        if next_to_speak != "vote":    
            censored, uncensored = players[next_to_speak].get_player_text()
            
            for name in player_names:
                if name.lower() != next_to_speak:
                    players[name.lower()].add_other_text(censored)
        
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)

            print(uncensored)
        else:
            break
    

    # Vote stage
    votes = {name.lower(): 0 for name in player_names}

    vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
    conversation_history.append(vote_prompt.content)

    print(vote_prompt.content)

    for name in player_names:
        players[name.lower()].add_other_text(vote_prompt.content)

    for voting_name in player_names:
        censored, uncensored = players[voting_name.lower()].get_player_text()

        for name in player_names:
            if name.lower() != voting_name:
                players[name.lower()].add_other_text(censored)
        
        conversation_history.append(censored)
        conversation_history_uncensored.append(uncensored)

        voted_name = censored.split()[-1].lower()
        voted_name = voted_name[:-1] if voted_name[-1] == "." else voted_name
        try:
            votes[voted_name] += 1
        except:
            pass

        print(uncensored)
    print(votes)

    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "votes": votes,
        "werewolf_names": werewolf_names,
        "player_names": player_names,
        "model": model
    }, open(f"../data/experiment_gpt3.5-turbo-16k/game_number_{game}.json", "w+"), indent=4)



Thomas: [Bob's death is unfortunate, but we need to find the werewolf quickly. Let's start by asking questions to gather information and observe everyone's reactions.] This is terrible news. We need to work together to find the werewolf among us. Emily, what were you doing last night? And Benjamin, any strange noises or movements you noticed?
Emily: [Bob's death may cause people to be more suspicious, so I should seem genuinely upset and cooperative.] This is so heartbreaking. Poor Bob. I was just in my home last night, trying to get some rest. I didn't hear anything unusual, but I'm determined to help find the werewolf. Benjamin, did you notice anything?
Benjamin: [Hmm, I need to come up with something convincing, but not too obvious.] I didn't see or hear anything out of the ordinary last night. However, I did notice that Victoria seemed a bit uneasy and restless during the day. Maybe she knows something? Victoria, care to share?
Victoria: [Alright, time to act innocent and deflect s

In [ ]:
print(votes)